In [1]:
import orjson
from IPython.display import Javascript, display, HTML, JSON
from ipycallback import SlowTunnelWidget
from labext.prelude import A as LEA
from sm_unk.prelude import D
from sm_unk.config import WIKIDATA_DIR
from sm_annotator.annotator import Annotator
from sm_unk.dev.wikitable2wikidata.prelude import *
from kg_data.prelude import redis, wikidata
%load_ext autoreload
%autoreload 2

In [2]:
tbl_store = redis.DBPediaTableStore.get_instance()
qnode_store = redis.WikiDataStore.get_instance()
ontclasses_store = redis.WDClassStore.get_instance()
ontprops_store = redis.WDPropertyStore.get_instance()

In [4]:
dataset_dir = HOME_DIR / "wikitable2wikidata/tables/s2_devset"
# dataset_dir = HOME_DIR / "wikitable2wikidata/ethiopia_tables"

In [5]:
# predictions = M.deserialize_pkl(dataset_dir / "predictions.pkl")
predictions = M.deserialize_pkl(dataset_dir / "predictions.pkl")

In [18]:
from urllib.parse import urlparse, unquote_plus
from hashlib import md5

def get_table_id(table_id: str):
    id = urlparse(table_id).path.replace("/resource/", "")
    return id + "_" + md5(table_id.encode()).hexdigest()

assert len(predictions) == len({get_table_id(p['tbl'].table.metadata.table_id) for p in predictions})
for pred in tqdm(predictions):
    table_id = get_table_id(pred['tbl'].table.metadata.table_id)
    outfile = dataset_dir / f"gold_predictions/{table_id}/version.01.json"
    outfile.parent.mkdir(exist_ok=True, parents=True)
    M.serialize_json({
        "sm": pred['sm'].to_json(),
        "is_curated": False,
        "note": ""
    }, outfile)

100%|██████████| 500/500 [00:00<00:00, 2163.03it/s]


In [23]:
annotator = Annotator(qnode_store, ontclasses_store, ontprops_store, dataset_dir / "gold_predictions", "http://localhost:9200", 'rook', 'discover@!')

def view(i, cache={}):
    sm, w2wtable = [predictions[i][k] for k in ['sm', 'tbl']]
    if not cache.get('is_rendered', False):
        annotator.render(dev=True)
        cache['is_rendered'] = True
        
    annotator.set_table(get_table_id(w2wtable.table.metadata.table_id), w2wtable)

LEA.slider(view, min=0, max=len(predictions) - 1, clear_output=False)

Output()

Run the following cells to set debug data

In [12]:
tunnel = SlowTunnelWidget(tunnel_id="development")
display(tunnel)
display(Javascript("""
var msg = window.recordTunnel.serialize();
window.IPyCallback.get("development").send_msg(msg);
"""))

SlowTunnelWidget(js_endpoint=(0, ''), py_endpoint=(0, ''), tunnel_id='development')

<IPython.core.display.Javascript object>

In [13]:
import time, json
for i in range(5):
    msg = tunnel.py_endpoint[1]
    if msg == '':
        print(".", end="")
        time.sleep(0.1)
        continue
        
    with open("./src/replayDebugData.ts", "w") as f:
        f.write("export const history = ")
        f.write(json.dumps(json.loads(msg), indent=4))

In [24]:
sm, w2wtable = [predictions[2][k] for k in ['sm', 'tbl']]

In [26]:
M.serialize_pkl((sm, w2wtable), "./data.pkl")

In [ ]:
with open("./data.json", "w") as f:
    f.write()